In [1]:
import pandas as pd
import os
import requests
from io import StringIO
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm

In [2]:
state_abv = {
    '2': 'AK',
    '1': 'AL',
    '5': 'AR',
    '4': 'AZ',
    '6': 'CA',
    '8': 'CO',
    '9': 'CT',
    '11': 'DC',
    '10': 'DE',
    '12': 'FL',
    '13': 'GA',
    '15': 'HI',
    '19': 'IA',
    '16': 'ID',
    '17': 'IL',
    '18': 'IN',
    '20': 'KS',
    '21': 'KY',
    '22': 'LA',
    '25': 'MA',
    '24': 'MD',
    '23': 'ME',
    '26': 'MI',
    '27': 'MN',
    '29': 'MO',
    '28': 'MS',
    '30': 'MT',
    '37': 'NC',
    '38': 'ND',
    '31': 'NE',
    '33': 'NH',
    '34': 'NJ',
    '35': 'NM',
    '32': 'NV',
    '36': 'NY',
    '39': 'OH',
    '40': 'OK',
    '41': 'OR',
    '42': 'PA',
    '44': 'RI',
    '45': 'SC',
    '46': 'SD',
    '47': 'TN',
    '48': 'TX',
    '49': 'UT',
    '51': 'VA',
    '50': 'VT',
    '53': 'WA',
    '55': 'WI',
    '54': 'WV',
    '56': 'WY'}

In [3]:
test_state_abv = {
    '2': 'AK',
    '1': 'AL',
    '5': 'AR',
    '4': 'AZ'}

In [4]:
comstock_buildings = ['quickservicerestaurant',
                      'fullservicerestaurant',
                      'smalloffice',
                      'mediumoffice',
                      'largeoffice',
                      'warehouse',
                      'smallhotel',
                      'largehotel',
                      'outpatient',
                      'hospital',
                      'secondaryschool',
                      'primaryschool',
                      'retailstandalone',
                      'retailstripmall']

In [5]:
test_comstock_buildings = ['smalloffice']

In [6]:
#upgrades = ['17','18']
upgrades = ['5','6','7','8','9','10','15','17','18']
#test_upgrades = ['17']

In [7]:
def plot_histogram_with_stats_and_fit(df):
    
    up = df['upgrade'].unique()[0]
    building = df['in.comstock_building_type'].unique()[0]
    state = df['in.state'].unique()[0]
    up = f"{int(up):02}"  # Convert string to int and format with leading zero
    column_name = 'calc.percent_savings.site_energy.total.energy_consumption_intensity..percent'
    
    # Count the number of buildings with a value greater than 25% / 50% in the specified column
    num_buildings_above_25_percent = (df[column_name] > 0.25).sum()
    num_buildings_above_50_percent = (df[column_name] > 0.5).sum()

    # Determine the total number of buildings
    total_buildings = df.shape[0]

    # Calculate the percentage of buildings that satisfy the condition
    percentage_above_25_percent = (num_buildings_above_25_percent / total_buildings) * 100
    percentage_above_50_percent = (num_buildings_above_50_percent / total_buildings) * 100

    print(f"Percentage of {building} with savings > 25%: {percentage_above_25_percent:.2f}%  > 50%: {percentage_above_50_percent:.2f}%")

     # Filter the DataFrame to include only rows where 'applicability' is TRUE
    # and create a copy to avoid SettingWithCopyWarning when modifying the DataFrame
    filtered_df = df[df['applicability'] == True].copy()
    
    # Use .loc[] to safely modify the specific column of interest
    data_series = filtered_df.loc[:, column_name] * 100
    
    mean_val = data_series.mean()
    median_val = data_series.median()
    std_val = data_series.std()
    min_val = data_series.min()
    max_val = data_series.max()

    # Fitting a normal distribution
    mu, std = norm.fit(data_series)

    # Plotting the histogram
    plt.figure(figsize=(10, 6))
    n, bins, patches = plt.hist(data_series, bins=30, color='blue', edgecolor='black', density=True)
    
    # Plotting the PDF of the fitted normal distribution
    xmin, xmax = plt.xlim()
    x = np.linspace(xmin, xmax, 100)
    p = norm.pdf(x, mu, std)
    plt.plot(x, p, 'k', linewidth=2)

    # Adding summary statistics to the plot
    plt.axvline(mean_val, color='red', linestyle='dashed', linewidth=1)
    plt.axvline(median_val, color='green', linestyle='dashed', linewidth=1)
    plt.legend({'Mean': mean_val, 'Median': median_val, 'Normal Fit': ''})

    plt.title(f'Histogram with Stats of {column_name}')
    plt.xlabel(column_name)
    plt.ylabel('Density')

    # Displaying summary statistics
    plt.text(xmin, max(n)*0.8, f'Mean: {mean_val:.2f}\nMedian: {median_val:.2f}\nStd: {std_val:.2f}\nMin: {min_val:.2f}\nMax: {max_val:.2f}', 
             bbox=dict(facecolor='white', alpha=0.5))

    #plt.show()
    
    # Construct the local file path
    directory = f'annual/plots/{up}/{building}'
    os.makedirs(directory, exist_ok=True)  # Ensure the directory exists
    file_path = f'{directory}/{up}_{building}_{state.upper()}_total_energy_percent_savings.png'
        
    plt.savefig(file_path, bbox_inches='tight', dpi=300)
    #plt.show()
    # Return the calculated percentages
    return percentage_above_25_percent, percentage_above_50_percent

In [8]:
columns=[
    'upgrade',
    'in.comstock_building_type',
    'in.state',
    'applicability',
    'in.sqft',
    'in.year_built',
    'in.energy_code_followed_during_last_ext_lighting_replacement',
    'in.energy_code_followed_during_last_hvac_replacement',
    'in.energy_code_followed_during_last_int_equipment_replacement',
    'in.energy_code_followed_during_last_roof_replacement',
    'in.energy_code_followed_during_last_svc_water_htg_replacement',
    'in.energy_code_followed_during_last_walls_replacement',
    'in.energy_code_followed_during_original_building_construction',
    'calc.percent_savings.electricity.total.energy_consumption_intensity..percent',
    'calc.percent_savings.electricity.total.energy_consumption..percent',
    'calc.percent_savings.natural_gas.total.energy_consumption_intensity..percent',
    'calc.percent_savings.natural_gas.total.energy_consumption..percent',
    'calc.percent_savings.site_energy.total.energy_consumption..percent',
    'calc.percent_savings.site_energy.total.energy_consumption_intensity..percent'
]

In [9]:
base_url = "https://oedi-data-lake.s3.amazonaws.com/nrel-pds-building-stock/end-use-load-profiles-for-us-building-stock/2023/comstock_amy2018_release_2/metadata_and_annual_results/by_state/state={STATE}/csv/{STATE}_upgrade{up}_metadata_and_annual_results.csv"
#https://oedi-data-lake.s3.amazonaws.com/nrel-pds-building-stock/end-use-load-profiles-for-us-building-stock/2023/comstock_amy2018_release_2/metadata_and_annual_results/by_state/state=CO/csv/CO_upgrade18_metadata_and_annual_results.csv

In [ ]:
# loop over upgrades
for upgrade in upgrades:
    UP = str(upgrade)
    up = f"{int(upgrade):02}"  # Convert string to int and format with leading zero

    data_df = pd.DataFrame(columns=['upgrade', 'state', 'comstock_building_type', 'percentage_above_25', 'percentage_above_50'])

    # Loop through each state abbreviation
    for state in state_abv.values():
        # Construct the local file path
        directory = f'annual/data/{up}'
        os.makedirs(directory, exist_ok=True)  # Ensure the directory exists
        file_path = f'{directory}/{state.upper()}_upgrade{up}_metadata_and_annual_results.csv'

        # Check if the file exists locally
        if os.path.exists(file_path):
            print(f"Using local file for upgrade: {up}, state: {state}")
            df = pd.read_csv(file_path, low_memory=False)
        else:
            # If the file doesn't exist, construct the URL and download the file
            url = base_url.format(up=up, STATE=state.upper())
            print(f"URL: {url}")
            try:
                response = requests.get(url)
                if response.status_code == 200:
                    print(f"Downloading upgrade: {up}, state: {state}")
                    # Convert the CSV content to a DataFrame
                    csv_content = StringIO(response.content.decode('utf-8'))
                    df = pd.read_csv(csv_content, low_memory=False)
                    
                    # Save the DataFrame locally for future use
                    df.to_csv(file_path, index=False)
                    print(f"Saved {file_path}")
                else:
                    print(f"Failed to download data for {state.upper()}: HTTP {response.status_code}")
            except Exception as e:
                print(f"Error downloading data for {state.upper()}: {e}")          


        # Group by 'in.comstock_building_type'
        grouped = df.groupby('in.comstock_building_type')

        # Iterate over each group, filter columns that match the defined columns, and save as separate files
        for building_type, group in grouped:
            # Filter columns that match the defined columns
            filtered_df = group[columns].dropna(how='all')  # Optionally, remove rows where all selected columns are NaN

            #make histogram plot and compute %buildings above 25% savings
            percentage_above_25, percentage_above_50 = plot_histogram_with_stats_and_fit(filtered_df)
            new_data = {
                'upgrade': [up],
                'state': [state.upper()],
                'comstock_building_type': [building_type],
                'percentage_above_25': [percentage_above_25],
                'percentage_above_50': [percentage_above_50]
            }

            # add new entry to DF
            new_row_df = pd.DataFrame(new_data)
            data_df = pd.concat([data_df, new_row_df], ignore_index=True)
    print(f'saving {up}_data.csv')                
    data_df.to_csv(f'annual/data/{up}/{up}_data.csv', index=False)          